# Voxel Domain Orchestrator

## Purpose

This notebook teaches you how to use VoxelDomainClient for high-level orchestration of complete AM-QADF workflows. You'll learn automated workflows, storage management, batch processing, and workflow orchestration with interactive controls.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Use VoxelDomainClient for orchestration
- ✅ Create, load, process, and store voxel grids
- ✅ Perform batch operations on multiple grids
- ✅ Manage voxel grid storage (MongoDB, GridFS, File)
- ✅ Automate complete workflows
- ✅ Monitor orchestration progress and status

## Estimated Duration

60-90 minutes

---

## Overview

VoxelDomainClient is the main orchestrator of the AM-QADF framework, providing a high-level interface for:

- 🎛️ **Orchestration**: Automated workflow management
- 🧊 **Grid Management**: Create, load, process, store voxel grids
- 💾 **Storage**: MongoDB, GridFS, and file-based storage
- 🔄 **Batch Processing**: Process multiple grids efficiently
- 📊 **Workflow Automation**: Complete pipeline automation

Use the interactive widgets below to orchestrate voxel domain operations - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import VoxelDomainClient
try:
    from am_qadf.voxel_domain import VoxelDomainClient
    from am_qadf.voxel_domain.storage import VoxelGridStorage
    ORCHESTRATOR_AVAILABLE = True
except ImportError as e:
    ORCHESTRATOR_AVAILABLE = False
    print(f"⚠️ VoxelDomainClient not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
⚠️ VoxelDomainClient not available: No module named 'am_qadf.voxel_domain.storage' - using demo mode
✅ Setup complete!


## Interactive Voxel Domain Orchestrator Interface

Use the widgets below to orchestrate voxel domain operations. Select orchestration mode, configure parameters, and monitor progress interactively!


In [2]:
# Create Interactive Voxel Domain Orchestrator Interface

# Global state
orchestration_state = {
    'current_operation': None,
    'grid_id': None,
    'grid_metadata': {},
    'operation_results': {}
}

# ============================================
# Top Panel: Orchestration Mode and Actions
# ============================================

orchestration_mode = RadioButtons(
    options=[
        ('Create', 'create'),
        ('Load', 'load'),
        ('Process', 'process'),
        ('Store', 'store'),
        ('Batch', 'batch')
    ],
    value='create',
    description='Mode:',
    style={'description_width': 'initial'}
)

grid_label = widgets.HTML("<b>Grid:</b>")
grid_selector = Dropdown(
    options=[('New Grid', 'new'), ('Grid 001', 'grid001'), ('Grid 002', 'grid002')],
    value='new',
    description='Grid:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Orchestration',
    button_style='success',
    icon='play',
    layout=Layout(width='200px')
)

save_config_button = Button(
    description='Save Configuration',
    button_style='primary',
    icon='save',
    layout=Layout(width='180px')
)

top_panel = HBox([
    orchestration_mode,
    grid_label,
    grid_selector,
    execute_button,
    save_config_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Orchestration Configuration
# ============================================

# Create Configuration
create_grid_type = Dropdown(
    options=[('Uniform', 'uniform'), ('Adaptive', 'adaptive'), ('Multi-Resolution', 'multi_res')],
    value='uniform',
    description='Grid Type:',
    style={'description_width': 'initial'}
)
create_resolution = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Resolution:', style={'description_width': 'initial'})
create_bbox_min_x = FloatSlider(value=-50, min=-100, max=0, step=1, description='BBox Min X:', style={'description_width': 'initial'})
create_bbox_max_x = FloatSlider(value=50, min=0, max=100, step=1, description='BBox Max X:', style={'description_width': 'initial'})
create_sources = SelectMultiple(
    options=[('Laser', 'laser'), ('ISPM', 'ispm'), ('CT', 'ct'), ('Hatching', 'hatching')],
    value=['laser', 'ispm'],
    description='Sources:',
    style={'description_width': 'initial'}
)

create_config = VBox([
    widgets.HTML("<b>Create Configuration:</b>"),
    create_grid_type,
    create_resolution,
    create_bbox_min_x,
    create_bbox_max_x,
    create_sources
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Load Configuration
load_storage_type = RadioButtons(
    options=[('MongoDB', 'mongodb'), ('GridFS', 'gridfs'), ('File', 'file')],
    value='mongodb',
    description='Storage:',
    style={'description_width': 'initial'}
)
load_grid_id = Text(value='grid001', description='Grid ID:', style={'description_width': 'initial'})
load_signals = Checkbox(value=True, description='Load Signals', style={'description_width': 'initial'})
load_metadata = Checkbox(value=True, description='Load Metadata', style={'description_width': 'initial'})
load_quality = Checkbox(value=False, description='Load Quality', style={'description_width': 'initial'})

load_config = VBox([
    widgets.HTML("<b>Load Configuration:</b>"),
    load_storage_type,
    load_grid_id,
    load_signals,
    load_metadata,
    load_quality
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Process Configuration
process_steps = SelectMultiple(
    options=[('Map Signals', 'map'), ('Align Data', 'align'), ('Correct Data', 'correct'), 
             ('Fuse Data', 'fuse'), ('Assess Quality', 'quality')],
    value=['map', 'align', 'fuse'],
    description='Steps:',
    style={'description_width': 'initial'}
)
process_method = Dropdown(
    options=[('Nearest', 'nearest'), ('Linear', 'linear'), ('IDW', 'idw')],
    value='nearest',
    description='Method:',
    style={'description_width': 'initial'}
)

process_config = VBox([
    widgets.HTML("<b>Process Configuration:</b>"),
    process_steps,
    process_method
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Store Configuration
store_storage_type = RadioButtons(
    options=[('MongoDB', 'mongodb'), ('GridFS', 'gridfs'), ('File', 'file')],
    value='mongodb',
    description='Storage:',
    style={'description_width': 'initial'}
)
store_compress = Checkbox(value=True, description='Compress', style={'description_width': 'initial'})
store_index = Checkbox(value=True, description='Create Index', style={'description_width': 'initial'})
store_metadata = Checkbox(value=True, description='Store Metadata', style={'description_width': 'initial'})

store_config = VBox([
    widgets.HTML("<b>Store Configuration:</b>"),
    store_storage_type,
    store_compress,
    store_index,
    store_metadata
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Batch Configuration
batch_size = IntSlider(value=10, min=1, max=1000, step=1, description='Batch Size:', style={'description_width': 'initial'})
batch_workers = IntSlider(value=4, min=1, max=100, step=1, description='Workers:', style={'description_width': 'initial'})
batch_source = Dropdown(
    options=[('MongoDB Query', 'mongodb'), ('File List', 'file')],
    value='mongodb',
    description='Source:',
    style={'description_width': 'initial'}
)

batch_config = VBox([
    widgets.HTML("<b>Batch Configuration:</b>"),
    batch_size,
    batch_workers,
    batch_source
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Dynamic configuration accordion
config_accordion = Accordion(children=[
    create_config,
    load_config,
    process_config,
    store_config,
    batch_config
])
config_accordion.set_title(0, 'Create')
config_accordion.set_title(1, 'Load')
config_accordion.set_title(2, 'Process')
config_accordion.set_title(3, 'Store')
config_accordion.set_title(4, 'Batch')

left_panel = VBox([
    widgets.HTML("<h3>Orchestration Configuration</h3>"),
    config_accordion
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Orchestration Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Workflow', 'workflow'), ('Status', 'status'), ('Results', 'results'), ('Storage', 'storage')],
    value='workflow',
    description='View:',
    style={'description_width': 'initial'}
)

viz_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Orchestration Visualization</h3>"),
    viz_mode,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Orchestration Status
# ============================================

# Current Operation
current_op_label = widgets.HTML("<b>Current Operation:</b>")
current_op_display = widgets.HTML("No operation running")
current_op_section = VBox([
    current_op_label,
    current_op_display
], layout=Layout(padding='5px'))

# Grid Information
grid_info_label = widgets.HTML("<b>Grid Information:</b>")
grid_info_display = widgets.HTML("No grid loaded")
grid_info_section = VBox([
    grid_info_label,
    grid_info_display
], layout=Layout(padding='5px'))

# Storage Information
storage_info_label = widgets.HTML("<b>Storage Information:</b>")
storage_info_display = widgets.HTML("No storage info available")
storage_info_section = VBox([
    storage_info_label,
    storage_info_display
], layout=Layout(padding='5px'))

# Batch Status
batch_status_label = widgets.HTML("<b>Batch Status:</b>")
batch_status_display = widgets.HTML("No batch operation")
batch_status_section = VBox([
    batch_status_label,
    batch_status_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_grid_button = Button(description='Export Grid', button_style='', layout=Layout(width='150px'))
export_config_button = Button(description='Export Config', button_style='', layout=Layout(width='150px'))
export_report_button = Button(description='Export Report', button_style='', layout=Layout(width='150px'))
save_orch_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_grid_button,
    export_config_button,
    export_report_button,
    save_orch_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    current_op_section,
    grid_info_section,
    storage_info_section,
    batch_status_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to orchestrate")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
log_display = Output(layout=Layout(height='100px', overflow='auto'))

bottom_panel = VBox([
    status_display,
    progress_bar,
    log_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Orchestration Functions
# ============================================

def execute_orchestration(button):
    """Execute orchestration operation."""
    global orchestration_state
    
    mode = orchestration_mode.value
    status_display.value = f"<b>Status:</b> Executing {mode} operation..."
    progress_bar.value = 0
    
    with log_display:
        clear_output(wait=True)
        print(f"Starting {mode} orchestration...")
    
    try:
        if mode == 'create':
            execute_create()
        elif mode == 'load':
            execute_load()
        elif mode == 'process':
            execute_process()
        elif mode == 'store':
            execute_store()
        elif mode == 'batch':
            execute_batch()
        
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ {mode.capitalize()} operation completed</span>"
        
        with log_display:
            print(f"✅ {mode.capitalize()} operation completed successfully")
        
        update_displays()
        update_visualization()
        
    except Exception as e:
        with log_display:
            print(f"❌ Error: {str(e)}")
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during {mode} operation</span>"
        progress_bar.value = 0

def execute_create():
    """Execute create operation."""
    global orchestration_state
    
    progress_bar.value = 20
    with log_display:
        print("Creating voxel grid...")
    
    # Simulate grid creation
    import time
    time.sleep(0.5)
    
    grid_id = f"grid_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    orchestration_state['grid_id'] = grid_id
    orchestration_state['grid_metadata'] = {
        'grid_type': create_grid_type.value,
        'resolution': create_resolution.value,
        'sources': list(create_sources.value),
        'created_at': datetime.now().isoformat()
    }
    
    progress_bar.value = 100
    with log_display:
        print(f"✅ Grid created: {grid_id}")

def execute_load():
    """Execute load operation."""
    global orchestration_state
    
    progress_bar.value = 20
    with log_display:
        print(f"Loading grid from {load_storage_type.value}...")
    
    import time
    time.sleep(0.5)
    
    grid_id = load_grid_id.value
    orchestration_state['grid_id'] = grid_id
    orchestration_state['grid_metadata'] = {
        'grid_id': grid_id,
        'storage_type': load_storage_type.value,
        'loaded_at': datetime.now().isoformat()
    }
    
    progress_bar.value = 100
    with log_display:
        print(f"✅ Grid loaded: {grid_id}")

def execute_process():
    """Execute process operation."""
    global orchestration_state
    
    progress_bar.value = 20
    with log_display:
        print("Processing voxel grid...")
        print(f"Steps: {', '.join(process_steps.value)}")
    
    import time
    steps = list(process_steps.value)
    for i, step in enumerate(steps):
        time.sleep(0.3)
        progress_bar.value = 20 + int(60 * (i + 1) / len(steps))
        with log_display:
            print(f"  ✅ {step} completed")
    
    orchestration_state['operation_results']['processing'] = {
        'steps': steps,
        'method': process_method.value,
        'completed_at': datetime.now().isoformat()
    }
    
    progress_bar.value = 100

def execute_store():
    """Execute store operation."""
    global orchestration_state
    
    progress_bar.value = 20
    with log_display:
        print(f"Storing grid to {store_storage_type.value}...")
    
    import time
    time.sleep(0.5)
    
    orchestration_state['operation_results']['storage'] = {
        'storage_type': store_storage_type.value,
        'compressed': store_compress.value,
        'indexed': store_index.value,
        'stored_at': datetime.now().isoformat()
    }
    
    progress_bar.value = 100
    with log_display:
        print(f"✅ Grid stored to {store_storage_type.value}")

def execute_batch():
    """Execute batch operation."""
    global orchestration_state
    
    progress_bar.value = 10
    with log_display:
        print(f"Starting batch operation (size: {batch_size.value}, workers: {batch_workers.value})...")
    
    import time
    batch_items = batch_size.value
    
    for i in range(batch_items):
        time.sleep(0.1)
        progress_bar.value = 10 + int(80 * (i + 1) / batch_items)
        if (i + 1) % 5 == 0:
            with log_display:
                print(f"  Processed {i + 1}/{batch_items} items")
    
    orchestration_state['operation_results']['batch'] = {
        'batch_size': batch_items,
        'workers': batch_workers.value,
        'completed': batch_items,
        'failed': 0,
        'completed_at': datetime.now().isoformat()
    }
    
    progress_bar.value = 100
    with log_display:
        print(f"✅ Batch operation completed: {batch_items} items processed")

def update_displays():
    """Update status displays."""
    global orchestration_state
    
    mode = orchestration_mode.value
    
    # Current operation
    if orchestration_state.get('current_operation'):
        op_html = f"<p><b>Type:</b> {orchestration_state['current_operation']}</p>"
        op_html += f"<p><b>Status:</b> Running</p>"
    else:
        op_html = f"<p><b>Type:</b> {mode.capitalize()}</p>"
        op_html += "<p><b>Status:</b> Ready</p>"
    current_op_display.value = op_html
    
    # Grid information
    if orchestration_state.get('grid_id'):
        grid_html = f"<p><b>Grid ID:</b> {orchestration_state['grid_id']}</p>"
        if orchestration_state.get('grid_metadata'):
            meta = orchestration_state['grid_metadata']
            grid_html += f"<p><b>Type:</b> {meta.get('grid_type', 'N/A')}</p>"
            grid_html += f"<p><b>Resolution:</b> {meta.get('resolution', 'N/A')}</p>"
            grid_html += f"<p><b>Sources:</b> {', '.join(meta.get('sources', []))}</p>"
    else:
        grid_html = "<p>No grid loaded</p>"
    grid_info_display.value = grid_html
    
    # Storage information
    if orchestration_state.get('operation_results', {}).get('storage'):
        storage = orchestration_state['operation_results']['storage']
        storage_html = f"<p><b>Type:</b> {storage['storage_type']}</p>"
        storage_html += f"<p><b>Compressed:</b> {storage['compressed']}</p>"
        storage_html += f"<p><b>Indexed:</b> {storage['indexed']}</p>"
        storage_html += f"<p><b>Size:</b> ~50 MB</p>"
    else:
        storage_html = "<p>No storage info available</p>"
    storage_info_display.value = storage_html
    
    # Batch status
    if mode == 'batch' and orchestration_state.get('operation_results', {}).get('batch'):
        batch = orchestration_state['operation_results']['batch']
        batch_html = f"<p><b>Completed:</b> {batch['completed']}</p>"
        batch_html += f"<p><b>Failed:</b> {batch['failed']}</p>"
        batch_html += f"<p><b>Workers:</b> {batch['workers']}</p>"
        batch_html += f"<p><b>Progress:</b> {batch['completed']/batch['batch_size']*100:.1f}%</p>"
    else:
        batch_html = "<p>No batch operation</p>"
    batch_status_display.value = batch_html

def update_visualization():
    """Update visualization display."""
    global orchestration_state
    
    with viz_output:
        clear_output(wait=True)
        
        viz = viz_mode.value
        mode = orchestration_mode.value
        
        if viz == 'workflow':
            # Workflow visualization
            fig, ax = plt.subplots(1, 1, figsize=(14, 8))
            ax.set_xlim(0, 12)
            ax.set_ylim(0, 10)
            ax.axis('off')
            
            # Draw workflow based on mode
            if mode == 'create':
                steps = ['Query Data', 'Create Grid', 'Map Signals', 'Store Grid']
            elif mode == 'load':
                steps = ['Load Grid', 'Load Signals', 'Load Metadata']
            elif mode == 'process':
                steps = ['Map Signals', 'Align Data', 'Fuse Data', 'Assess Quality']
            elif mode == 'store':
                steps = ['Prepare Grid', 'Compress', 'Store', 'Create Index']
            else:  # batch
                steps = ['Load Batch', 'Process Items', 'Store Results']
            
            for i, step in enumerate(steps):
                x = 2
                y = 8 - i * 1.2
                color = '#c8e6c9' if i < len(steps) - 1 else '#e3f2fd'
                
                box = FancyBboxPatch((x-0.5, y-0.3), 2.0, 0.6, 
                                    boxstyle="round,pad=0.1", 
                                    facecolor=color, edgecolor='black', linewidth=2)
                ax.add_patch(box)
                ax.text(x+0.5, y, step, ha='center', va='center', 
                       fontsize=10, fontweight='bold')
                
                if i < len(steps) - 1:
                    arrow = FancyArrowPatch((x+0.5, y-0.4), (x+0.5, y-0.9),
                                           arrowstyle='->', mutation_scale=20,
                                           color='black', linewidth=1.5)
                    ax.add_patch(arrow)
            
            ax.text(6, 9.5, f'{mode.capitalize()} Orchestration Workflow', 
                   ha='center', va='center', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.show()
        
        elif viz == 'status':
            # Status visualization
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Progress chart
            if orchestration_state.get('operation_results'):
                results = orchestration_state['operation_results']
                if 'batch' in results:
                    batch = results['batch']
                    axes[0].bar(['Completed', 'Failed'], 
                               [batch['completed'], batch['failed']],
                               color=['green', 'red'], alpha=0.7)
                    axes[0].set_ylabel('Count')
                    axes[0].set_title('Batch Operation Status')
                    axes[0].grid(True, alpha=0.3, axis='y')
                else:
                    axes[0].text(0.5, 0.5, 'No batch operation', 
                               ha='center', va='center', transform=axes[0].transAxes)
                    axes[0].set_title('Operation Status')
            else:
                axes[0].text(0.5, 0.5, 'No operation data', 
                           ha='center', va='center', transform=axes[0].transAxes)
                axes[0].set_title('Operation Status')
            
            # Resource usage (simulated)
            axes[1].bar(['CPU', 'Memory', 'Storage'], [60, 45, 30],
                       color=['blue', 'green', 'orange'], alpha=0.7)
            axes[1].set_ylabel('Usage (%)')
            axes[1].set_title('Resource Usage')
            axes[1].set_ylim(0, 100)
            axes[1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'results':
            # Results visualization
            if orchestration_state.get('grid_id'):
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                
                # Simulate grid visualization
                x = np.linspace(0, 10, 50)
                y = np.linspace(0, 10, 50)
                X, Y = np.meshgrid(x, y)
                Z = np.sin(X) * np.cos(Y)
                
                contour = ax.contourf(X, Y, Z, levels=20, cmap='plasma')
                ax.set_xlabel('X')
                ax.set_ylabel('Y')
                ax.set_title(f'Grid Visualization: {orchestration_state["grid_id"]}')
                plt.colorbar(contour, ax=ax, label='Signal Value')
                plt.tight_layout()
                plt.show()
            else:
                display(HTML("<p>No grid available for visualization</p>"))
        
        elif viz == 'storage':
            # Storage visualization
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))
            
            # Simulate storage structure
            storage_types = ['MongoDB', 'GridFS', 'File']
            sizes = [30, 50, 20]  # MB
            colors = ['#e3f2fd', '#c8e6c9', '#fff3e0']
            
            ax.pie(sizes, labels=storage_types, autopct='%1.1f%%', 
                  colors=colors, startangle=90)
            ax.set_title('Storage Distribution')
            plt.tight_layout()
            plt.show()

# Update configuration visibility based on mode
def update_config_visibility(change):
    """Update which configuration section is visible."""
    mode = change['new']
    
    # Show relevant accordion section
    config_accordion.selected_index = {
        'create': 0,
        'load': 1,
        'process': 2,
        'store': 3,
        'batch': 4
    }.get(mode, 0)
    
    # Update current operation
    orchestration_state['current_operation'] = mode
    update_displays()

orchestration_mode.observe(update_config_visibility, names='value')

# Connect events
execute_button.on_click(execute_orchestration)
viz_mode.observe(lambda x: update_visualization(), names='value')
orchestration_mode.observe(lambda x: update_visualization(), names='value')

# Initial updates
update_displays()
update_visualization()

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to use VoxelDomainClient for orchestration.

### Key Takeaways

1. **Orchestration Modes**: Create, Load, Process, Store, Batch operations
2. **Create Configuration**: Grid type, resolution, bounding box, data sources
3. **Load Configuration**: Storage type (MongoDB/GridFS/File), grid ID, load options
4. **Process Configuration**: Processing pipeline steps, method selection
5. **Store Configuration**: Storage type, compression, indexing, metadata
6. **Batch Configuration**: Batch size, parallel workers, batch source
7. **Workflow Visualization**: Interactive workflow diagrams for each mode
8. **Status Monitoring**: Real-time operation status, progress, resource usage
9. **Storage Management**: MongoDB, GridFS, and file-based storage options

### Next Steps

Proceed to:
- **19_Advanced_Analytics_Workflow.ipynb** - Advanced analytics and research workflows

### Related Resources

- Voxel Domain Documentation: `../docs/AM_QADF/05-modules/voxel-domain.md`
- API Reference: `../docs/AM_QADF/06-api-reference/voxel-domain-api.md`
- Examples: `../examples/`
